# Every-visit MC prediction 


 
(c) Fabrice Mulotti

In [1]:
import gymnasium as gym
import pandas as pd
from collections import defaultdict

Create a blackjack environment:

In [2]:
env = gym.make('Blackjack-v1')

## Définissons une politque

state[0] renvoie le nombre de point du joueur.   

L'action :      
0 est standby   
1 est hit (nouvelle carte)  

In [3]:
def policy(pstate):
    return 0 if pstate[0] > 19 else 1

In [4]:
## créons notre environnement
state = env.reset()
print(state)

((12, 10, 1), {})


In [5]:
print(policy(state[0]))

1


In [6]:
# De devrait pas renvoyer d'erreur
assert policy([19,2,2]) == 1

In [7]:
# Devrait renvoyer une erreur
assert policy([20,2,2]) == 1

AssertionError: 

***
## Generatrice d'épisode
Notre prochain fonction va nous permettre de générer des épisodes


In [8]:
def generate_episode(policy,max_turn):
    
    # Enregistrement des épisodes
    episode = []
    
    # Remise à zéro env
    ge_state = env.reset()[0]
  
    # déroulement de la partie
    for i in range(max_turn):
        # choix d'une action en fonction de notre police
        action = policy(ge_state)
        
        # on joue notre action en on récupère les résultats
        ge_next_state, reward, done, truncated, info = env.step(action)
        
        # enregistrement
        episode.append((ge_state, action, reward))
        
        # Condition de fin ?
        if done or truncated:
            break

        # sinon la partie continue
        ge_state = ge_next_state

    return episode

In [9]:
# Devrait produit une partie
generate_episode(policy,100)

[((14, 10, 0), 1, 0.0), ((19, 10, 0), 1, 0.0), ((21, 10, 0), 0, 0.0)]

***
## Calcul de la fonction de valeur

Nous allons devoir générer un grand nombre d'épisode et ajuster la `valeur moyenne de v` à chaque passage sur un état s

 

In [27]:
# total_return = fonction de valeur pour chaque état
total_return = defaultdict(float)

# N nombre de passage sur chaque état
N = defaultdict(int)

In [28]:
# Nombre d'itération
num_iterations = 100000
gamma=0.90

In [29]:
for i in range(num_iterations):
    
    # générons un épisode
    episode = generate_episode(policy,100)

    # stockons l'état, action et récompense obtenue / zip permet de prendre chaque 1, 2eme, 3eme.. terme
    states, actions, rewards = zip(*episode)
    # pour chaque état rencontré dans l'épisode

    G=0
    # print(states)
    for t in range(len(states)-1,-1,-1):
        state=states[t]
        # print(f"t={t}, state={state}, R={R}")
        G=G*gamma+rewards[t] 
        # if state not in states[t:]:
        total_return[state] += G
        N[state] += 1
            

***
## Analysons nos résultats
Convertissons les données en dataframe pour une lecture plus facile

In [30]:
total_return = pd.DataFrame(total_return.items(),columns=['state', 'total_return'])

In [31]:
N = pd.DataFrame(N.items(),columns=['state', 'N'])

Merge des deux dataframes

In [32]:
df = pd.merge(total_return, N, on="state")

Résultat : 

In [33]:
df.head(10)

,state,total_return,N
0,"(20, 10, 1)",364.00000,787
1,"(18, 10, 0)",-3145.75000,4334
2,"(14, 10, 0)",-2379.28690,3872
3,"(20, 1, 1)",29.00000,196
4,"(21, 10, 0)",2712.00000,3050
5,"(15, 10, 0)",-2494.41190,3925
6,"(12, 10, 0)",-2010.77500,3574
7,"(14, 10, 1)",-110.15019,404
8,"(12, 7, 0)",-460.69560,929
9,"(21, 5, 1)",478.00000,489


Calculons la fonction de valeur.
A ce stade nous avons tous les éléments (cumul des G et nombre de passage N)

In [34]:
df['value'] = df['total_return']/df['N']

In [36]:
df.head(20)

,state,total_return,N,value
0,"(20, 10, 1)",364.00000,787,0.462516
1,"(18, 10, 0)",-3145.75000,4334,-0.725831
2,"(14, 10, 0)",-2379.28690,3872,-0.614485
3,"(20, 1, 1)",29.00000,196,0.147959
4,"(21, 10, 0)",2712.00000,3050,0.889180
5,"(15, 10, 0)",-2494.41190,3925,-0.635519
6,"(12, 10, 0)",-2010.77500,3574,-0.562612
7,"(14, 10, 1)",-110.15019,404,-0.272649
8,"(12, 7, 0)",-460.69560,929,-0.495905
9,"(21, 5, 1)",478.00000,489,0.977505


***
## Analyse

A ce stade nous pouvons apprécier la valeur de notre politique (seuil à 19).
Exemple dans une situation ou nous avons 21 points, et par exemple le croupier 9 points 



In [41]:
# sans AS
df[df['state']==(21,10,False)]['value'].values

array([0.88918033])

In [42]:
# avec AS
df[df['state']==(21,10,True)]['value'].values

array([0.90568119])

La fonction de valeur est proche de 1. Donc efficiente.



---
Regardons maintenant dans un contexte plus difficile avec une main prochain de 19 (voir politique)

In [43]:
df[df['state']==(19,8,False)]['value'].values

array([-0.77940905])

In [44]:
df[df['state']==(19,8,True)]['value'].values

array([-0.31996112])

La valeur est moindre car l'incertitude de la suite de la partie plus grande